Visão geral do código
Abaixo está um pipeline completo, pronto para Colab, que:
- Carrega imagens em pastas por classe.
- Extrai embeddings com uma CNN pré-treinada (ResNet50).
- Indexa os vetores com FAISS para busca rápida.
- Retorna recomendações visuais com base na similaridade.
- Mostra resultados lado a lado.
Se você já tiver um dataset organizado como data/<classe>/*.jpg, basta ajustar o caminho


In [ ]:
# No Colab, rode isso
!pip install faiss-cpu --quiet
!pip install tensorflow==2.15.0 --quiet
!pip install opencv-python --quiet

import os
import glob
import numpy as np
import faiss
import cv2
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image

Preparação dos dados

In [ ]:
# Ajuste para o seu dataset
DATA_DIR = "/content/data"  # ex.: /content/data/camiseta, /content/data/relógio, etc.
IMG_SIZE = (224, 224)

def list_images_by_class(data_dir):
    classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    paths, labels = [], []
    for c in classes:
        for p in glob.glob(os.path.join(data_dir, c, "*")):
            if p.lower().endswith((".jpg", ".jpeg", ".png")):
                paths.append(p)
                labels.append(c)
    return paths, labels, classes

image_paths, image_labels, classes = list_images_by_class(DATA_DIR)
print(f"Total de imagens: {len(image_paths)} | Classes: {classes}")

Modelo de embeddings com ResNet50

In [ ]:
# Modelo pré-treinado sem a camada de classificação, saída em vetor de 2048
base_model = ResNet50(weights="imagenet", include_top=False, pooling="avg")

def load_and_preprocess(img_path, target_size=IMG_SIZE):
    img = image.load_img(img_path, target_size=target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)  # preprocess do ResNet50
    return x

def compute_embedding(img_path):
    x = load_and_preprocess(img_path)
    emb = base_model.predict(x, verbose=0)
    return emb.squeeze()  # shape (2048,)

# Extrai embeddings de todo o dataset
embeddings = []
for p in tqdm(image_paths, desc="Extraindo embeddings"):
    emb = compute_embedding(p)
    embeddings.append(emb)
embeddings = np.vstack(embeddings).astype('float32')  # FAISS usa float32
print("Shape dos embeddings:", embeddings.shape)  # (N, 2048)

Indexação e busca com FAISS

In [ ]:
# Normaliza L2 para similaridade de cosseno via índice de dot-product
def l2_normalize(x):
    norms = np.linalg.norm(x, axis=1, keepdims=True) + 1e-10
    return x / norms

embeddings_norm = l2_normalize(embeddings)

# Índice FAISS de produto interno (IP) para simular cosine similarity com vetores normalizados
d = embeddings_norm.shape[1]
index = faiss.IndexFlatIP(d)
index.add(embeddings_norm)  # adiciona todos os vetores

# Busca: retorna top-k vizinhos mais semelhantes
def search_similar(img_path, k=6):
    q = compute_embedding(img_path).astype('float32').reshape(1, -1)
    q = l2_normalize(q)
    scores, idxs = index.search(q, k)
    return idxs[0], scores[0]

# Utilitário para visualização
def show_results(query_path, idxs, scores, cols=3):
    imgs = [query_path] + [image_paths[i] for i in idxs]
    titles = ["Consulta"] + [f"{image_labels[i]} | score={scores[j]:.3f}" for j, i in enumerate(idxs)]
    rows = int(np.ceil(len(imgs)/cols))
    plt.figure(figsize=(4*cols, 4*rows))
    for i, p in enumerate(imgs):
        img = cv2.imread(p)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(rows, cols, i+1)
        plt.imshow(img)
        plt.axis("off")
        plt.title(titles[i], fontsize=10)
    plt.tight_layout()
    plt.show()

Consulta de exemplo e recomendações

In [ ]:
# Escolha uma imagem de consulta do seu dataset
query_image = image_paths[0]  # ou defina um caminho específico

idxs, scores = search_similar(query_image, k=6)
show_results(query_image, idxs, scores)

Alternativa sem FAISS usando scikit-learn

In [ ]:
!pip install scikit-learn --quiet
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=6, metric="cosine")
nn.fit(embeddings)

def search_sklearn(img_path, k=6):
    q = compute_embedding(img_path).reshape(1, -1)
    distances, idxs = nn.kneighbors(q, n_neighbors=k)
    # Similaridade ~ 1 - distância (cosine)
    scores = 1 - distances[0]
    return idxs[0], scores

idxs_s, scores_s = search_sklearn(query_image, k=6)
show_results(query_image, idxs_s, scores_s)